## This demo app shows:
* how to run Llama2 locally on a Mac using llama-cpp-python and the llama-cpp's quantized Llama2 model;
* how to use LangChain to ask Llama general questions;
* how to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination. 

In [1]:
# install all the required packages for the demo
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install pypdf sentence-transformers chromadb langchain

In [2]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [4]:
# create the Llama2 model - for more info see https://python.langchain.com/docs/integrations/llms/llamacpp
llm = LlamaCpp(
    model_path="<path-to-ggml-model-q4_0.gguf>"
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager, 
    verbose=True,
)

llama_model_loader: loaded meta data with 16 key-value pairs and 363 tensors from /Users/jeffxtang/repos/llama/llama-2-13b-chat/ggml-model-q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q6_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_gate.weight 

In [5]:
# the simplest way to ask Llama some general questions
question = "who wrote the book Innovator's dilemma?"
answer = llm(question)



The book "The Innovator's Dilemma" was written by Clayton Christensen, a professor at Harvard Business School. It was first published in 1997 and has since become a widely influential book on business strategy and innovation.


llama_print_timings:        load time =  1202.24 ms
llama_print_timings:      sample time =    46.44 ms /    58 runs   (    0.80 ms per token,  1249.03 tokens per second)
llama_print_timings: prompt eval time =  1815.15 ms /    15 tokens (  121.01 ms per token,     8.26 tokens per second)
llama_print_timings:        eval time =  5582.64 ms /    57 runs   (   97.94 ms per token,    10.21 tokens per second)
llama_print_timings:       total time =  7545.78 ms


In [6]:
# a more flexible way to ask Llama general questions using LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    "who wrote {book}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("innovator's dilemma")

Llama.generate: prefix-match hit




Clayton Christensen is the author of "The Innovator's Dilemma," which was first published in 1997. The book explores why successful companies often struggle to adapt to disruptive technologies and business models that ultimately lead to their downfall. Christensen argues that these companies are faced with a dilemma because they have invested so heavily in their existing products and processes that it is difficult for them to pivot and embrace new, disruptive technologies. He also introduces the concept of "disruptive innovation," which he defines as a process by which a small company with limited resources is able to successfully challenge established industry leaders.


llama_print_timings:        load time =  1202.24 ms
llama_print_timings:      sample time =   116.69 ms /   147 runs   (    0.79 ms per token,  1259.79 tokens per second)
llama_print_timings: prompt eval time =  1180.31 ms /     8 tokens (  147.54 ms per token,     6.78 tokens per second)
llama_print_timings:        eval time = 13192.98 ms /   147 runs   (   89.75 ms per token,    11.14 tokens per second)
llama_print_timings:       total time = 14746.13 ms


In [7]:
# let's see how Llama2 hallucinates, because it doesn't have the knowledge about Llama2 while the model was trained, 
# but by default it behaves like a know-it-all expert who can't afford to say I don't know
prompt = PromptTemplate.from_template(
    "What is {what}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("llama2")

Llama.generate: prefix-match hit




Llama2 is a free, open-source tool for generating high-quality, randomized test data for software applications. It is designed to be easy to use and to produce realistic, diverse test data that can help you identify and fix bugs in your application before it is released.

Llama2 is the successor to the popular Llama tool, and it includes many new features and improvements over its predecessor. Some of the key features of Llama2 include:

* Support for a wide range of data types, including strings, numbers, dates, and more
* The ability to generate random data based on user-defined rules and constraints
* A powerful and flexible API that allows you to customize and extend the tool to meet your specific needs
* Integration with popular testing frameworks and tools, such as JUnit and TestNG
* Support for a variety of programming languages, including Java, Python, C#, and more.

Overall, Llama2 is a powerful and flexible tool that can help you improve the quality and reliability of your 


llama_print_timings:        load time =  1202.24 ms
llama_print_timings:      sample time =   191.25 ms /   240 runs   (    0.80 ms per token,  1254.87 tokens per second)
llama_print_timings: prompt eval time =   480.79 ms /     6 tokens (   80.13 ms per token,    12.48 tokens per second)
llama_print_timings:        eval time = 22013.19 ms /   239 runs   (   92.11 ms per token,    10.86 tokens per second)
llama_print_timings:       total time = 23111.55 ms


In [8]:
# to fix the LLM's hallucination, one way is to use RAG, to augment it with more recent or custom data that holds the info for it to answer correctly
# first load the Llama2 paper via the LangChain's PDF loader
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("llama2.pdf")
documents = loader.load()

In [9]:
# quick check on the loaded document for the correct pages etc
print(len(documents), documents[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


In [10]:
# there're more 30 vector stores (DBs) supported by LangChain. Chroma is light-weight and in memory so it's easy to get started with
# other vector stores can be used to store large amount of data - see https://python.langchain.com/docs/integrations/vectorstores
from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
# split the loaded documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

In [12]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

# for each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context
# the Llama to answer question about the data stored in the verctor db
question = "What is llama2?"
result = qa_chain({"query": question})
# it takes close to 2 minutes to return the result (but using other vector store than Chroma such as FAISS can take longer), because 
# Llama2 is running on a local Mac. To get much faster results, you can use a cloud service with GPU used for inference - see HelloLlamaCloud 
# for a demo.

Llama.generate: prefix-match hit


 Llama 2 is a new language model developed by Meta AI that has been released openly to encourage responsible AI innovation. It is a fine-tuned version of the original Llama model and is optimized for dialogue use cases. The model has not covered all scenarios and may produce inaccurate or objectionable responses, so developers should perform safety testing and tuning before deploying any applications of Llama 2.


llama_print_timings:        load time =  1202.24 ms
llama_print_timings:      sample time =    76.83 ms /    97 runs   (    0.79 ms per token,  1262.48 tokens per second)
llama_print_timings: prompt eval time = 97067.98 ms /  1146 tokens (   84.70 ms per token,    11.81 tokens per second)
llama_print_timings:        eval time = 10431.81 ms /    96 runs   (  108.66 ms per token,     9.20 tokens per second)
llama_print_timings:       total time = 107897.31 ms
